In [3]:
!pip install tensorboard==2.2.0 --use-feature=2020-resolver
!pip install tensorflow_text>=2.0.0rc0 --use-feature=2020-resolver
!pip install -U sentence-transformers --use-feature=2020-resolver

ERROR: pytorch-lightning 0.9.0 requires tensorboard==2.2.0, but you'll have tensorboard 2.3.0 which is incompatible.
     |████████████████████████████████| 59 kB 1.6 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 7.2 MB/s eta 0:00:01
     |████████████████████████████████| 3.0 MB 33.8 MB/s eta 0:00:01


  Created wheel for sentence-transformers: filename=sentence_transformers-0.3.7.2-py3-none-any.whl size=91186 sha256=9c306f1dac5aaaff9ddc219e767199ca79f0c0aafd0ed77e756ae1d985b7c77e
  Stored in directory: /root/.cache/pip/wheels/df/42/15/b8329fd622ed2e9ee15b00314a387aa7d70e402d144349d540
Successfully built sentence-transformers
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.8.1rc1
    Uninstalling tokenizers-0.8.1rc1:
      Successfully uninstalled tokenizers-0.8.1rc1
  Attempting uninstall: transformers
    Found existing installation: transformers 3.0.2
    Uninstalling transformers-3.0.2:
      Successfully uninstalled transformers-3.0.2
ERROR: allennlp 1.1.0 requires transformers<3.1,>=3.0, but you'll have transformers 3.3.1 which is incompatible.


In [4]:
import json
import numpy as np
import pandas as pd
import tensorflow_hub as hub
import tensorflow_text
import torch
from sentence_transformers import SentenceTransformer, util, models

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [5]:
with open('../input/queries/queries.json') as json_file:
    data = json.load(json_file)
data

{'Names': [{'Name': 'Хоз-питьевой водопровод В1',
   'Items': [{'Item': 'Фланец плоский стальной свободный DN150 PN1,0 МПа',
     'Model': '',
     'Manufacturer': '',
     'Quantity': '11',
     'Measure': 'шт.'},
    {'Item': 'Упор бетонный',
     'Model': '',
     'Manufacturer': '',
     'Quantity': '3',
     'Measure': 'шт.'},
    {'Item': 'Гидрант пожарный подземный Н-1,5',
     'Model': '',
     'Manufacturer': '',
     'Quantity': '3',
     'Measure': 'шт.'}]},
  {'Name': 'Водопровод хозяйственно-питьевой В1',
   'Items': [{'Item': 'Задвижка AVK фланцевая короткая DN80 с ответными фланцами и крепежом',
     'Model': '',
     'Manufacturer': '',
     'Quantity': '2',
     'Measure': 'шт.'},
    {'Item': 'Труба ПЭ 100 SDR 13,6-560х41,2 техническая',
     'Model': '',
     'Manufacturer': '',
     'Quantity': '42,0',
     'Measure': 'м'},
    {'Item': 'Теплоизоляция ROCKWOOL TEX MAT 100 мм, с уплот. по толщ. 1, 2',
     'Model': '',
     'Manufacturer': '',
     'Quantity': '10,5'

In [6]:
queries = [item['Item'] for project in data['Names'] for item in project['Items']]

In [7]:
handbook = pd.read_excel('../input/handbook/handbook.xls')
handbook.head()

,Код,Артикул,Наименование,Полное наименование,Вид номенклатуры,Базовая единица измерения,Единица хранения остатков,Номер ГТД,Страна происхождения,Ставка НДС,Номенклатурная группа,Ценовая группа,Комментарий
0,1920,NaN,"500х23,9 труба ПНД ""техническая""","Труба ПНД ""техническая"" 500х23,9",Товар,м,м,NaN,NaN,20%,NaN,NaN,NaN
1,УТ000004069,NaN,003L0280 RLV-K R1/2-G3/4 Узел подключения для ...,003L0280 RLV-K R1/2-G3/4 Узел подключения для ...,Товар,шт,шт,NaN,NaN,20%,NaN,NaN,NaN
2,УТ000000195,NaN,003L0280 Клапан RLV-K G 3/4A; G 1/2А,"003L0280 Н-образный клапан RLV-K, прямой, для ...",Товар,шт,шт,NaN,NaN,20%,NaN,NaN,NaN
3,УТ000000198,NaN,003L7603 Регулятор перепада давления ASV-PV Ду25,003L7603 Регулятор перепада давления ASV-PV с ...,Товар,шт,шт,NaN,NaN,20%,NaN,NaN,NaN
4,УТ000001434,NaN,003L7604 Регулятор перепада давления ASV-PV Ду40,003L7604 Регулятор перепада давления ASV-PV с ...,Товар,шт,шт,NaN,NaN,20%,NaN,NaN,NaN


In [8]:
# 2 или 3
print(handbook.iloc[:, 2].isnull().values.any())
print(handbook.iloc[:, 3].isnull().values.any())

False
True


In [9]:
handbook.iloc[:, 2] = handbook.iloc[:, 2].fillna('')

In [10]:
handbook.duplicated().any()

False

## Multilingual BERT for sentences

In [11]:
# Use BERT for mapping tokens to embeddings
word_embedding_model = models.BERT('distilbert-base-multilingual-cased')

# Apply mean pooling to get one fixed sized sentence vector
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                               pooling_mode_mean_tokens=True,
                               pooling_mode_cls_token=False,
                               pooling_mode_max_tokens=False)

bert_model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

In [12]:
# Corpus with example sentences
handbook_names = handbook.iloc[:, 2].tolist()

In [13]:
handbook_bert_embeddings = bert_model.encode(handbook_names, convert_to_tensor=True)

In [14]:
# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
top_k = 5

In [15]:
for query in queries:
    query_embedding = bert_model.encode(query, convert_to_tensor=True)
    cos_scores = util.pytorch_cos_sim(query_embedding, handbook_bert_embeddings)[0]
    cos_scores = cos_scores.cpu()
    
    top_results = torch.topk(cos_scores, k=top_k)
    
    print("Query:", query)
    print("\nTop 5 most similar sentences:")
    # print(handbook.iloc[top_results.indices])
    for score, idx in zip(top_results[0], top_results[1]):
        print(handbook_names[idx], "(Score: %.4f)" % (score))


Query: Фланец плоский стальной свободный DN150 PN1,0 МПа

Top 5 most similar sentences:
Фланец плоский стальной Ду80 (90тр) PN16 (Score: 0.9108)
Фланец стальной плоский Ду 100 PN16 (Score: 0.8955)
Фланец плоский стальной Ду65 PN16 (Score: 0.8906)
Фланец Maxidaptor DN350 OD374,5-391,5  PN10 универсальный для труб из стали, чугуна, ПВХ (Score: 0.8844)
Колодец пластиковый (ПЭ) Wosser, ∅1000, SN2, Нср=1,66м, (дождеприемный) (Score: 0.8831)



Query: Упор бетонный

Top 5 most similar sentences:
Упор бетонный УГ1 (Score: 0.8997)
Упор бетонный УН29 (Score: 0.8819)
Клапан запорный прямой 1/2" (Score: 0.7833)
Ремонтный комплект тормозных накладок (Score: 0.7768)
Трубы напорные внутрянка (Score: 0.7707)



Query: Гидрант пожарный подземный Н-1,5

Top 5 most similar sentences:
Гидрант пожарный Н 1,5 DN100 (Score: 0.9398)
Гидрант пожарный Н 1,5 м DN125 (Score: 0.9361)
Гидрант пожарный Н 1,25 м DN125 (Score: 0.9342)
Гидрант пожарный Н 2,5 м DN125 (Score: 0.9337)
Гидрант пожарный Н 2,75 м DN125 (Score: 0.9319)



Query: Задвижка AVK фланцевая короткая DN80 с ответными фланцами и крепежом

Top 5 most similar sentences:
Задвижка клиновая фланцевая DN200 PN10 короткая AVK (Score: 0.9380)
Задвижка клиновая фланцевая DN400 PN10 короткая AVK (Score: 0.9373)
Задвижка клиновая фланцевая DN350 PN10 короткая AVK (Score: 0.9323)
Задвижка клиновая фланцевая DN600 PN10 короткая AVK (Score: 0.9311)
Задвижка клиновая фланцевая DN300 PN10 короткая AVK (Score: 0.9309)



Query: Труба ПЭ 100 SDR 13,6-560х41,2 техническая

Top 5 most similar sentences:
110х4,2 труба SDR26 "техническая" (Score: 0.9470)
110х6,3 труба SDR17.6 "техническая" бухта 50м   (Score: 0.9444)
160х6,2 труба SDR26 "техническая" отр.12м (Score: 0.9441)
160х11,8 труба SDR13.6 "техническая" отр.12м   (Score: 0.9391)
560х21,4 труба ПНД "техническая" (Score: 0.9378)



Query: Теплоизоляция ROCKWOOL TEX MAT 100 мм, с уплот. по толщ. 1, 2

Top 5 most similar sentences:
Теплоизоляция Цилиндр ТехноНИКОЛЬ 80 (1,2м/шт) кашированные 57х30 (Score: 0.8940)
Теплоизоляция ENERGOFLEX SUPER 110/9 2 м (Score: 0.8897)
Труба  32х2,0 электросварная ГОСТ 1074-91 с антикор. покр. внут. AMERCOAT 391 PC, нар. ВУС коснтр. 4 (Score: 0.8892)
Труба 108х4,0 электросварная ГОСТ 1074-91 с антикор. покр. внут. AMERCOAT 391 PC, нар. ВУС коснтр. 4 (Score: 0.8880)
Теплоизоляция ENERGOFLEX SUPER 76/25 2 м (Score: 0.8873)



Query: Фильтрующий модуль для очистки сточных вод

Top 5 most similar sentences:
Фильтр топливный грубой очистки (Score: 0.8923)
Цоколь для терморегулятора  (Score: 0.8676)
Аренда прижимного устройства для седельных отводов (Score: 0.8624)
Муфты для термокабеля Nelson (Score: 0.8618)
Модуль фильтрующий "Эковод" ФМС-1,0 (Score: 0.8614)


## Universal Sentence Encoder Multilingual

In [17]:
use_embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")

# Compute embeddings.
use_embeddings = use_embed(handbook_names)

In [18]:
# Compute similarity matrix. Higher score indicates greater similarity.

# convert_to_tensor
for query in queries:
    query_embedding = use_embed(query)
    similarity_matrix = np.inner(use_embeddings, query_embedding)
    top_results = (-similarity_matrix).argsort(axis=0)[:top_k].ravel()
    
    print('\nQuery:', query)
    print('\nTop 5 most similar sentences:')
    # print(handbook.iloc[top_results])
    for idx in top_results:
        print(handbook_names[idx], '(Score: %.4f)' % (similarity_matrix[idx]))


Query: Фланец плоский стальной свободный DN150 PN1,0 МПа

Top 5 most similar sentences:
Фланец стальной плоский Ду 100 PN16 (Score: 0.7419)
Задвижка чугунная фланцевая PN 10  МЗВ DN150 (Score: 0.6848)
Подставка под ПГ ППТФ - 150х100 сталь (Score: 0.6826)
Подставка под ПГ ППФ - 150 сталь (Score: 0.6700)
Подставка под ПГ ППФО - 150 сталь (Score: 0.6648)

Query: Упор бетонный

Top 5 most similar sentences:
Упор бетонный УН29 (Score: 0.8105)
Упор бетонный УГ1 (Score: 0.7732)
Круг зачист. бетон. (Score: 0.7622)
Смесь цементно-песчаная, 25кг (Score: 0.4228)
Диск алм. 115х22,23 мм по по бетону (Score: 0.4213)

Query: Гидрант пожарный подземный Н-1,5

Top 5 most similar sentences:
Гидрант пожарный Н 1,5 м (Score: 0.8760)
Гидрант пожарный Н 1,5 м JAFAR (Score: 0.8313)
Гидрант пожарный Н 1,5 DN100 (Score: 0.8130)
Гидрант пожарный Н 2,5 м (Score: 0.7846)
Гидрант пожарный Н 1,5 м DN125 (Score: 0.7685)

Query: Задвижка AVK фланцевая короткая DN80 с ответными фланцами и крепежом

Top 5 most similar